In [ ]:
import PKoopmanDL as pkdl
import numpy as np
import matplotlib.pyplot as plt
import torch

# read the config file
config_file = "kdv.json"
solver = pkdl.ParamKoopmanDLSolverWrapper(config_file)

# set up the grid
Nx = 128
x_max = torch.pi
x_min = -torch.pi
x_step = (x_max - x_min) / (Nx - 1)
x_grid = torch.linspace(x_min, x_max, Nx).unsqueeze(0)

def sample_func(row_size, col_size):
  assert (col_size == Nx)
  x = x_grid.expand(row_size, Nx)
  # torch.manual_seed(0)
  uniform = torch.tensor([1.0, 1.0, 1.0])
  b = torch.distributions.Dirichlet(uniform).sample((row_size,))
  b1 = b[:, 0].view(-1, 1)
  b2 = b[:, 1].view(-1, 1)
  b3 = b[:, 2].view(-1, 1)
  eta = b1 * torch.exp(-(x - torch.pi / 2)**2) - b2 * torch.sin(
      x / 2)**2 + b3 * torch.exp(-(x + torch.pi / 2)**2)
  return eta

def nontrain_func(x):
  one = torch.ones((x.shape[0], 1))
  mass = torch.sum(x, dim=1, keepdim=True) * x_step
  momentum = torch.sum(x **2, dim=1, keepdim=True) * x_step
  return torch.cat((one, mass, momentum), dim=1)
  

# set up the solver
solver.load_dataset_setup("data/kdv_dataset.pt", nontrain_func, sample_func)
K = solver.solve()

## Solve the Tracking Problem

In [ ]:
n_traj = solver.n_traj
t_step = solver.t_step
target_dim = 2
time_horizon = 10
lambda_param = 0

soln_ref = torch.zeros(n_traj, target_dim) + 0.3
state0 = solver.train_dataset.dataset.data_x[0]

for i in range(soln_ref.size(0)):
  if i > n_traj / 2:
    soln_ref[i, :] = 0.5

mass_ref = x_step * torch.sum(soln_ref, dim=-1, keepdim=True)
momentum_ref = x_step * torch.sum(soln_ref**2, dim=-1, keepdim=True)

mass_solver = pkdl.KoopmanMPCSolver(solver.dynamics, K, solver.dictionary, mass_ref, time_horizon, 1, lambda_param)
mass_solver.solve(state0, -1, 1)


